In [14]:
import os 
import re
import json
from groq import Groq
import datetime
from dotenv import load_dotenv

In [15]:
load_dotenv()

True

In [16]:
search_api_key = os.getenv("TAVILY_SEARCH_API")

In [17]:
# os.chdir("..")

In [18]:
base_data_path = r"C:\Users\Wayne\Documents\GitHub\MultiAgentResearcher\model_output_data"


if os.path.exists(base_data_path):
    print("It exists")
else:
    print("Does not exist")

It exists


In [19]:
folders = [
    os.path.join(base_data_path, f) # Gives me the path so I can run datetime on it and get the most recent folder 
    for f in os.listdir(base_data_path) # Grabs all the folders in it
    if os.path.isdir(os.path.join(base_data_path, f)) # Validates that the path actually exists
]


In [20]:
latest_folder = max(folders, key=os.path.getctime)
print("Latest folder:", latest_folder)


Latest folder: C:\Users\Wayne\Documents\GitHub\MultiAgentResearcher\model_output_data\research_on_java


## Temporary List to store data 

In [21]:
messages = []

In [22]:
tasks_path = os.path.join(latest_folder, "tasks.json")

with open(tasks_path, "r") as f:
    tasks = json.load(f)

In [23]:
retriever_agent = Groq()

In [24]:
from tavily import TavilyClient

tavily = TavilyClient(api_key=search_api_key)

def fetch_search_results(query):

    response = tavily.search(
        query=query,
        search_depth="basic",
        max_results=5
    )

    structured_results = []

    for result in response["results"]:
        structured_results.append({
            "title": result["title"],
            "url": result["url"],
            "snippet": result["content"][:500]  # limit tokens
        })

    return structured_results

In [25]:
retriever_system_prompt = """
You are a research retrieval agent.
Your job: Extract accurate facts from the provided search results for the following task:
Task: {task_description}

Rules:
- Use ONLY the given search results
- Do NOT invent information
- Keep summaries concise and factual
- Include the source URL for every item
- If results are insufficient, return an empty list

IMPORTANT: Your response must contain ONLY the <answer> block below. No extra text, no explanation, no preamble.

<answer>
[
  {{
    "source": "...",
    "title": "...",
    "summary": "...",
    "key_points": ["...", "..."]
  }}
]
</answer>

Do not write anything before or after the <answer> tags.
"""

messages.append({"role": "system", "content": retriever_system_prompt})

In [26]:
retriever_agent = Groq()

for task in tasks:

    task_description = task["description"]

    formatted_prompt = retriever_system_prompt.format(
        task_description=task_description
    )

    search_results = fetch_search_results(task_description)

    # Fresh message state per task
    messages = [
        {"role": "system", "content": formatted_prompt},
        {
            "role": "user",
            "content": f"Here are the search results:\n\n{search_results}"
        }
    ]

    completion = retriever_agent.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=messages,
        stream=False
    )

    reply = completion.choices[0].message.content

    print("Task:", task_description)
    print(reply)
    print("=" * 50)

Task: Investigate the history and evolution of the Java programming language
<answer>
[
  {
    "source": "https://www.theknowledgeacademy.com/blog/history-and-evolution-of-java/",
    "title": "History and Evolution of Java Programming Language",
    "summary": "Java, a versatile and influential programming language, has left an indelible mark on the world of software development.",
    "key_points": [
      "Java was developed by James Gosling and colleagues at Sun Microsystems in the early 1990s.",
      "Java is intended to be compiled to a bytecode, which is then run by a Java Virtual Machine.",
      "The first version of Java was released in 1995."
    ]
  },
  {
    "source": "https://www.freejavaguide.com/history.html",
    "title": "History of Java programming language | www.freejavaguide.com",
    "summary": "Java is an object-oriented programming language developed by James Gosling and colleagues at Sun Microsystems in the early 1990s.",
    "key_points": [
      "Java is d

In [27]:
from pprint import pprint

In [28]:
pprint(messages)

[{'content': '\n'
             'You are a research retrieval agent.\n'
             'Your job: Extract accurate facts from the provided search '
             'results for the following task:\n'
             'Task: Explore the most popular libraries, frameworks, and tools '
             'used in Java development\n'
             '\n'
             'Rules:\n'
             '- Use ONLY the given search results\n'
             '- Do NOT invent information\n'
             '- Keep summaries concise and factual\n'
             '- Include the source URL for every item\n'
             '- If results are insufficient, return an empty list\n'
             '\n'
             'IMPORTANT: Your response must contain ONLY the <answer> block '
             'below. No extra text, no explanation, no preamble.\n'
             '\n'
             '<answer>\n'
             '[\n'
             '  {\n'
             '    "source": "...",\n'
             '    "title": "...",\n'
             '    "summary": "...",\n'
  

In [29]:
agent_output = reply
agent_output

'<answer>\n[\n  {\n    "source": "https://medium.com/thefreshwrites/exploring-the-most-common-java-libraries-frameworks-6c7ae547d23c",\n    "title": "Exploring The Most Common Java Libraries & Frameworks",\n    "summary": "A list of popular Java libraries and frameworks.",\n    "key_points": ["Spring Framework", "Hibernate", "Apache Maven", "Guava", "JUnit", "Apache Commons"]\n  },\n  {\n    "source": "https://www.diffblue.com/resources/most-popular-libraries-used-by-java-developers/",\n    "title": "Most popular libraries used by Java developers 2025 - Diffblue",\n    "summary": "A list of popular Java libraries, enhancing productivity and simplifying complex tasks for developers.",\n    "key_points": ["Spring Framework", "Hibernate", "Apache Commons"]\n  },\n  {\n    "source": "https://scand.com/company/blog/top-open-source-java-frameworks-and-tools/",\n    "title": "Most Popular Open Source Java Frameworks and Tools - SCAND",\n    "summary": "A list of open-source Java frameworks an

In [30]:
start = agent_output.find("<answer>") + len("<answer>")
end = agent_output.find("</answer>")

json_text = agent_output[start:end].strip()

search_result = json.loads(json_text)

with open(os.path.join(latest_folder, "retrieval_results.json"), "w") as f:
    json.dump(search_result, f, indent=4)

print(search_result)

[{'source': 'https://medium.com/thefreshwrites/exploring-the-most-common-java-libraries-frameworks-6c7ae547d23c', 'title': 'Exploring The Most Common Java Libraries & Frameworks', 'summary': 'A list of popular Java libraries and frameworks.', 'key_points': ['Spring Framework', 'Hibernate', 'Apache Maven', 'Guava', 'JUnit', 'Apache Commons']}, {'source': 'https://www.diffblue.com/resources/most-popular-libraries-used-by-java-developers/', 'title': 'Most popular libraries used by Java developers 2025 - Diffblue', 'summary': 'A list of popular Java libraries, enhancing productivity and simplifying complex tasks for developers.', 'key_points': ['Spring Framework', 'Hibernate', 'Apache Commons']}, {'source': 'https://scand.com/company/blog/top-open-source-java-frameworks-and-tools/', 'title': 'Most Popular Open Source Java Frameworks and Tools - SCAND', 'summary': 'A list of open-source Java frameworks and tools, including NetBeans.', 'key_points': ['Spring', 'NetBeans', 'Eclipse']}, {'sour